In [ ]:
# pip install google-cloud-storage
# !pip install google-cloud-documentai google-cloud-storage pandas

In [9]:
pip install --upgrade google-cloud-documentai

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import time
import json
import pandas as pd
from google.cloud import documentai_v1 as documentai
from google.cloud.documentai_v1.types import Document, BatchInputConfig, BatchOutputConfig
from google.cloud import storage

# 🔧 Config
project_id = "vercillopersonal"
location = "us"
processor_id = "fe61eee8945a8018"
gcs_input_uri = "gs://vercillo_projects/transactions/amex/2025/2025-06-03.pdf"
gcs_output_uri = "gs://vercillo_projects/transactions/amex/exports/"
output_csv_path = "amex_2025_06_03.csv"

client = documentai.DocumentProcessorServiceClient()
name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

# 🧾 Set up GCS input/output
input_config = documentai.BatchProcessRequest.BatchInputConfig(
    gcs_source=gcs_input_uri,
    mime_type="application/pdf"
)
output_config = documentai.BatchProcessRequest.BatchOutputConfig(
    gcs_destination=gcs_output_uri
)

# 🛠️ Create the batch process request
request = documentai.BatchProcessRequest(
    name=name,
    input_configs=[input_config],
    output_config=output_config
)

# 🏃‍♂️ Start the batch process
operation = client.batch_process_documents(request)

print("⏳ Processing... this may take a moment")
operation.result(timeout=300)  # wait max 5 minutes

print("✅ Document processed and output stored in GCS")

# 🔄 Extract .json from output and read it
bucket_name = "vercillo_projects"
output_prefix = "transactions/amex/exports/"
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blobs = list(bucket.list_blobs(prefix=output_prefix))

# Find the output JSON (Document AI writes one or more)
json_blobs = [b for b in blobs if b.name.endswith(".json")]

if not json_blobs:
    raise FileNotFoundError("❌ No output JSON found in the specified GCS folder.")

# Download the first JSON
json_blob = json_blobs[0]
json_blob.download_to_filename("output.json")

# 📖 Parse Document object
from google.cloud.documentai_v1.types import Document
import json

with open("output.json", "r") as f:
    doc_data = json.load(f)

document = Document.from_json(json.dumps(doc_data))

# 📊 Extract entities
entities = []
for entity in document.entities:
    entities.append({
        "type": entity.type_,
        "value": entity.mention_text,
        "confidence": round(entity.confidence, 3),
        "page": entity.page_anchor.page_refs[0].page if entity.page_anchor.page_refs else None
    })

df = pd.DataFrame(entities)
df.to_csv(output_csv_path, index=False)

print(f"✅ Final CSV exported to: {output_csv_path}")


ImportError: cannot import name 'BatchInputConfig' from 'google.cloud.documentai_v1.types' (c:\Users\jverc\anaconda3\Lib\site-packages\google\cloud\documentai_v1\types\__init__.py)